# Data Generation

In [1]:
from random import seed
from random import randint

# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y
 
seed(1)
n_samples = 1
n_numbers = 2
largest = 10

# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)

[[3, 10]] [13]


In [3]:
from math import ceil
from math import log10

# convert data to strings
def to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
        
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr
 
seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)

[[3, 10]] [13]
[' 3+10'] ['13']


In [4]:
# integer encode strings
def integer_encode(X, y, alphabet):
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
        
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc
 
seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
X, y = integer_encode(X, y, alphabet)
print(X, y)

[[3, 10]] [13]
[' 3+10'] ['13']
[[11, 3, 10, 1, 0]] [[1, 3]]


In [5]:
# one hot encode
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
        
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
X, y = integer_encode(X, y, alphabet)
print(X, y)
# one hot encode
X, y = one_hot_encode(X, y, len(alphabet))
print(X, y)

[[3, 10]] [13]
[' 3+10'] ['13']
[[11, 3, 10, 1, 0]] [[1, 3]]
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [6]:
# generate an encoded dataset
def generate_data(n_samples, n_numbers, largest, alphabet):
    # generate pairs
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    
    # convert to strings
    X, y = to_string(X, y, n_numbers, largest)
    
    # integer encode
    X, y = integer_encode(X, y, alphabet)
    
    # one hot encode
    X, y = one_hot_encode(X, y, len(alphabet))
    
    # return as numpy arrays
    X, y = array(X), array(y)
    return X, y

Finally, we need to invert the encoding to convert the output vectors back into numbers so we can compare expected output integers to predicted integers.

In [7]:
# invert encoding
def invert(seq, alphabet):
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    strings = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [8]:
# define dataset
seed(1)
n_samples = 1000
n_numbers = 2
largest = 10
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
n_chars = len(alphabet)
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))

# Configure and Fit a seq2seq LSTM Model

In [18]:
from numpy import array
from math import sqrt
from numpy import argmax
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import RepeatVector

# define LSTM configuration
n_batch = 10
n_epoch = 30
# create LSTM
model = Sequential()
model.add(LSTM(100, input_shape=(n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50, return_sequences=True))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
# train LSTM
for i in range(n_epoch):
    X, y = generate_data(n_samples, n_numbers, largest, alphabet)
    print(i)
    model.fit(X, y, epochs=1, batch_size=n_batch)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 100)               45200     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 2, 100)            0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 2, 50)             30200     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 2, 12)             612       
Total params: 76,012
Trainable params: 76,012
Non-trainable params: 0
_________________________________________________________________
None
0
Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 1.9928 - acc: 0.3595
1
Epoch 1/1
1000/1000 [==============================] - 1s 630us/step - loss: 1.5123 - acc: 0.3710
2
Epoch 1/1
1000/1000 [==============================] - 1s 620us/s

# Evaluate on some samples

In [20]:
# evaluate on some new patterns
X, y = generate_data(n_samples, n_numbers, largest, alphabet)
result = model.predict(X, batch_size=n_batch, verbose=0)
# calculate error
input1 = [invert(x, alphabet) for x in X]
expected = [invert(x, alphabet) for x in y]
predicted = [invert(x, alphabet) for x in result]
# show some examples
for i in range(50):
    print('Input=%s, Expected=%s, Predicted=%s' % (input1[i], expected[i], predicted[i]))

Input= 10+4, Expected=14, Predicted=14
Input=  6+4, Expected=10, Predicted=10
Input=  4+8, Expected=12, Predicted=12
Input=  5+4, Expected= 9, Predicted= 9
Input= 4+10, Expected=14, Predicted=14
Input=  7+7, Expected=14, Predicted=14
Input=  5+2, Expected= 7, Predicted= 7
Input= 10+7, Expected=17, Predicted=17
Input=  9+5, Expected=14, Predicted=14
Input=  1+2, Expected= 3, Predicted= 3
Input=  4+6, Expected=10, Predicted=10
Input=  8+8, Expected=16, Predicted=16
Input=  5+3, Expected= 8, Predicted= 8
Input=  8+7, Expected=15, Predicted=15
Input=  2+9, Expected=11, Predicted=11
Input=  3+9, Expected=12, Predicted=12
Input=  7+8, Expected=15, Predicted=15
Input=  4+2, Expected= 6, Predicted= 6
Input=  6+9, Expected=15, Predicted=15
Input=  9+4, Expected=13, Predicted=13
Input=  3+6, Expected= 9, Predicted= 9
Input=  3+1, Expected= 4, Predicted= 4
Input=  9+3, Expected=12, Predicted=12
Input=  9+4, Expected=13, Predicted=13
Input=  9+4, Expected=13, Predicted=13
Input=  6+8, Expected=14,